In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

Toools

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]


Agent Execution 

In [3]:
import json
from json import JSONDecodeError
from typing import List, Union

from langchain_core.agents import AgentAction, AgentActionMessageLog, AgentFinish
from langchain_core.exceptions import OutputParserException
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
)
from langchain_core.outputs import ChatGeneration, Generation

from langchain.agents.agent import AgentOutputParser


class OpenAIFunctionsAgentOutputParser(AgentOutputParser):
    """Parses a message into agent action/finish.

    Is meant to be used with OpenAI models, as it relies on the specific
    function_call parameter from OpenAI to convey what tools to use.

    If a function_call parameter is passed, then that is used to get
    the tool and tool input.

    If one is not passed, then the AIMessage is assumed to be the final output.
    """

    @property
    def _type(self) -> str:
        return "openai-functions-agent"

    @staticmethod
    def _parse_ai_message(message: BaseMessage) -> Union[AgentAction, AgentFinish]:
        """Parse an AI message."""
        if not isinstance(message, AIMessage):
            raise TypeError(f"Expected an AI message got {type(message)}")

        function_call = message.additional_kwargs.get("function_call", {})

        if function_call:
            function_name = function_call["name"]
            try:
                if len(function_call["arguments"].strip()) == 0:
                    # OpenAI returns an empty string for functions containing no args
                    _tool_input = {}
                else:
                    # otherwise it returns a json object
                    _tool_input = json.loads(function_call["arguments"], strict=False)
            except JSONDecodeError:
                raise OutputParserException(
                    f"Could not parse tool input: {function_call} because "
                    f"the `arguments` is not valid JSON."
                )

            # HACK HACK HACK:
            # The code that encodes tool input into Open AI uses a special variable
            # name called `__arg1` to handle old style tools that do not expose a
            # schema and expect a single string argument as an input.
            # We unpack the argument here if it exists.
            # Open AI does not support passing in a JSON array as an argument.
            if "__arg1" in _tool_input:
                tool_input = _tool_input["__arg1"]
            else:
                tool_input = _tool_input

            content_msg = f"responded: {message.content}\n" if message.content else "\n"
            log = f"\nInvoking: `{function_name}` with `{tool_input}`\n{content_msg}\n"
            return AgentActionMessageLog(
                tool=function_name,
                tool_input=tool_input,
                log=log,
                message_log=[message],
            )

        return AgentFinish(
            return_values={"output": message.content}, log=str(message.content)
        )

    def parse_result(
        self, result: List[Generation], *, partial: bool = False
    ) -> Union[AgentAction, AgentFinish]:
        if not isinstance(result[0], ChatGeneration):
            raise ValueError("This output parser only works on ChatGeneration output")
        message = result[0].message
        return self._parse_ai_message(message)

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        raise ValueError("Can only parse messages")


In [10]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain.agents.format_scratchpad.openai_functions import (
    format_to_openai_function_messages,
)

# from langchain.agents.output_parsers.openai_functions import (
#     OpenAIFunctionsAgentOutputParser,
# )
from langchain_community.chat_models import ChatOllama
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# from langchain_openai import ChatOpenAI

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# Choose the LLM that will drive the agent
# llm = ChatOpenAI(model="gpt-4-turbo-preview")

# llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q6_K', tool_system_prompt_template='')

llm = OllamaFunctions(model='mistral:7b-instruct-v0.2-q8_0', )

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

# llm_with_tools = llm.bind(functions=[convert_to_openai_function(t) for t in tools])

llm_with_tools = llm.bind(
    functions=[
    {'name': 'tavily_search_results_json', 
     'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer     questions about current events. Input should be a search query.', 
     'parameters': {'type': 'object', 
                    'properties': 
                        {'query': {'description': 'search query to look up', 'type': 'string'}}, 
                        'required': ['query']}}
    ]
)


agent = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        )
    )
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [11]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [12]:
from langgraph.prebuilt import create_agent_executor

agent_executor = create_agent_executor(agent, tools)




In [13]:
from langchain.globals import set_debug

set_debug(True)
from langchain.globals import set_verbose

set_verbose(True)

In [14]:
agent_executor.invoke(
    {"input": "who is the winnner of the us open", "chat_history": []}
)

[chain/start] [1:chain:LangGraph] Entering Chain run with input:
{
  "input": ""
}
[pregel/step] Starting step 0 with 1 task. Next tasks:
- __start__({'chat_history': [], 'input': 'who is the winnner of the us open'})
[chain/start] [1:chain:LangGraph > 2:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 2:chain:__start__] s] Exiting Chain run with output:
{
  "output": "who is the winnner of the us open"
}
[chain/start] [1:chain:LangGraph > 3:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 3:chain:__start__] s] Exiting Chain run with output:
{
  "output": []
}
[chain/start] [1:chain:LangGraph > 4:chain:__start__] Entering Chain run with input:
{
  "input": "who is the winnner of the us open",
  "chat_history": []
}
[chain/end] [1:chain:LangGraph > 4:chain:__start__] s] Exiting Ch

KeyboardInterrupt: 